## 根据用户的输入生成图片prompt模块代码封装

In [ ]:
from openai import OpenAI 

import json
def json_parse(image_prompt):
    image_prompt = image_prompt.replace("```\n","")
    image_prompt = image_prompt.replace("\n```","")
    # 您提供的字符串
    data_str =image_prompt #completion.choices[0].message.content


    # 将字符串转换为JSON对象
    try:
        data_json = json.loads(data_str)
        return data_json
    except json.JSONDecodeError as e:
        print("字符串不是有效的JSON格式：", e)
        
def gen_text_to_image_prompt(user_input): 
    client = OpenAI(api_key="你的阶跃APIkey", base_url="https://api.stepfun.com/v1")

    completion = client.chat.completions.create(
      model="step-1-200k",
      messages=[
          {
              "role": "system",
              "content": "你是由阶跃星辰提供的带文字海报生成助手，你擅长中文，英文，以及多种其他语言的对话。在保证用户数据安全的前提下，你能对用户的问题和请求，作出快速和精准的回答。同时，你的回答和建议应该拒绝黄赌毒，暴力恐怖主义的内容",
          },
          {"role": "user", "content": "你是带文本海报生成智能助理. \
                                       1.根据用户输入内容生成做图需要的描述prompt \
                                       2.描述图片详细具体内容 \
                                       3.描述图片风格、颜色分布、光影效果、纹理、材质\
                                       4.给出图片具体尺寸、比例 \
                                       5.适合的配文,按行分句放到list里\
                                       6.输出英文结果\
                                       7.json格式输出生成结果：{\
                                                  \"prompt\":\"把上面文本生成图描述+具体信息描述+风格颜色光影纹理材质描述汇总到prompt\",\
                                                   \"text\":\"适合用户输入场景的配文\",\
                                                   \"ar\":\"图片长:宽比,数字:数字\",\
                                                   \"hw\":\"图长宽具体尺寸\",\
                                                   \"X\":\"文字所在坐标位置值X\,\
                                                   \"Y\":\"文字所在坐标位置值Y\,\
                                                   \"is_vertical\":\"文字排版是否竖排\"}"},
          {"role": "user", "content":user_input},
      ],
    )
    out_put_chinese = completion.choices[0].message.content
    completion = client.chat.completions.create(
      model="step-1-200k",
      messages=[
          {
              "role": "system",
              "content": "你是由阶跃星辰提供的带文字海报生成助手，你擅长中文，英文，以及多种其他语言的对话。在保证用户数据安全的前提下，你能对用户的问题和请求，作出快速和精准的回答。同时，你的回答和建议应该拒绝黄赌毒，暴力恐怖主义的内容",
          },
          {"role": "user", "content": out_put_chinese +"\n把上面中文输出翻译成英文,无关信息不要过多输出" },

      ],
    )
    out_put = completion.choices[0].message.content
    print(out_put)
    out_put = json_parse(out_put)
    return out_put

## 图片生成模块封装，根据上面生成的prompt调用图片生成后台服务生成图片

In [ ]:
from gradio_client import Client
def image_generation(image_prompt,ip_url):
    client = Client(ip_url)
    image_prompt = image_prompt["prompt"] + " --ar " + image_prompt["ar"]
    out_data = client.predict(image_prompt,"dpm-solver",14,4.5,0,True)
    # 图片地址
    image_path = out_data[0]
    return image_path

## 根据用户输入生成prompt中抽取出文字排版的信息对文字排版

In [ ]:
def auto_text_layout(inputext=[], x=0,y=0,row_spacing=100, col_spacing=80, vertical=True, font="SimSun", color=(255, 255, 255, 0)):
    if vertical:
        output=[] 
        y0=y
        for text in inputext:
            for char in list(text):
                output.append({
                    "content": char,
                    "position": (x, y),
                    "font": font,
                    "color": color
                })
                y +=   col_spacing
            y= y0
            x += row_spacing
    else:
        output=[]
        x0 = x
        for text in inputext:
            for char in list(text):
                output.append({
                    "content": char,
                    "position": (x, y),
                    "font": font,
                    "color": color
                })
                x += row_spacing
            x=x0
            y += col_spacing
    return output

## 根据生成图片，排版好的文字合成海报模块

In [ ]:
import os
import json
from PIL import Image, ImageDraw, ImageFont
import cv2

def text_image_blender_poster(image_url,auto_text,font_path ,font_size,output_dir,out_file_name):
    # 设置海报模板路径、文字内容、输出目录等参数
    template_path = image_url  # 海报模板路径

    text_lines = auto_text
    output_dir = output_dir  # 输出目录
    font_path =  font_path # 字体文件路径
    font_size = font_size  # 字体大小

    # 确保输出目录存在
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 加载模板图片
    template_image = Image.open(template_path)

    # 创建一个可以在Pillow中使用的字体对象
    font = ImageFont.truetype(font_path, font_size)

    # 创建一个可以在Pillow中使用的绘图对象
    draw = ImageDraw.Draw(template_image)

    # 在指定位置添加文字
    for line in text_lines:
        draw.text(line['position'], line['content'], font=font, fill=line['color'])

    # 保存处理后的图片
    output_path = os.path.join(output_dir, 'poster_with_multiple_lines.jpg')
    template_image.save(output_path)

    # 如果需要进行图层融合和模糊处理，可以使用OpenCV
    # 读取处理后的图片
    image = cv2.imread(output_path)

    # 这里可以添加OpenCV的图层融合和模糊处理代码
    # 例如，使用高斯模糊
    blurred_image = cv2.GaussianBlur(image, (5, 5), 0)

    # 保存模糊处理后的图片
    cv2.imwrite(os.path.join(output_dir, out_file_name), blurred_image)

    print("海报生成和处理完成。")

## 生成海报是否合格验证模块

In [ ]:
from dashscope import MultiModalConversation
import dashscope
dashscope.api_key ='把你申请的qwenvl api-key放这边'
def call_with_local_file(local_file_path):
    """Sample of use local file.
       linux&mac file schema: file:///home/images/test.png
       windows file schema: file://D:/images/abc.png
    """
    local_file_path1 = local_file_path#'file:///Users/**/output_posters/poster_with_multiple_lines.jpg'

    messages = [{
        'role': 'system',
        'content': [{
            'text': 'You are a helpful assistant.'
        }]
    }, {
        'role':
        'user',
        'content': [
            {
                'image': local_file_path1
            },
            {
                'text': '请描述这张图,这张图中文字放置的位置合理吗？符合审美需求吗？\
                        1.如果合理请回复是,并给出合理原因\
                        2.如果不合理给出理由和建议\
                        3.如果有建议请给出文字合适放置的坐标位置\
                        4.如果不合理给出字体大小建议\
                        5.如果不合理给出字体颜色建议\
                        6.json格式输出回答结果'
            },
        ]
    }]
    response = MultiModalConversation.call(model=MultiModalConversation.Models.qwen_vl_chat_v1, messages=messages)
    print(response)

## 把上面的模块串接成自动化流程

In [ ]:
#根据用户输入生成图片生成prompt
image_prompt = gen_text_to_image_prompt("5.1劳动节")

#根据ptompt生成图
ip_url ="http://0.0.0.0:6006"
image_url = image_generation(image_prompt,ip_url)

#文字排版
text_lines = image_prompt["text"]
vertical = False if image_prompt["is_vertical"]=='false' else True
x =  float(image_prompt["X"])
y =  float(image_prompt["Y"])
auto_text=auto_text_layout(inputext=text_lines, x=x,y=y,row_spacing=48, col_spacing=63, vertical=False, font="SimSun", color=(255, 255, 255, 0))
print(auto_text)

#根据生成图片+排版文字合成海报
image_url =image_url
auto_text = auto_text
font_size = 42
output_dir = 'output_posters'
font_path="/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf"
out_file_name = 'blurred_poster.jpg'
text_image_blender_poster(image_url,auto_text,font_path ,font_size,output_dir,out_file_name)

#生成海报是否合格验证模块
local_file_path ='file://'+'/Users/**/output_posters/poster_with_multiple_lines.jpg'
call_with_local_file(local_file_path)